In [1]:
import pandas as pd
import numpy as np
import ast

In [5]:
#importamos el csv
dfreview = pd.read_csv("G:/Mi unidad/SOYHENRY/CIENCIA DE DATOS/CURSO/PROYECTOS/PI1_ML 2.0/DATA_ORIGINAL/csv/reviews.csv")
#visualizamos el csv
dfreview.head(10)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
5,Wackky,http://steamcommunity.com/id/Wackky,"[{'funny': '', 'posted': 'Posted May 5, 2014.'..."
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,"[{'funny': '1 person found this review funny',..."
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,"[{'funny': '', 'posted': 'Posted July 24.', 'l..."
8,76561198089393905,http://steamcommunity.com/profiles/76561198089...,"[{'funny': '5 people found this review funny',..."
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."


In [8]:
dfreview.loc[4, "reviews"]

'[{\'funny\': \'3 people found this review funny\', \'posted\': \'Posted April 15, 2014.\', \'last_edited\': \'\', \'item_id\': \'211420\', \'helpful\': \'35 of 43 people (81%) found this review helpful\', \'recommend\': True, \'review\': \'Git gud\'}, {\'funny\': \'1 person found this review funny\', \'posted\': \'Posted December 23, 2013.\', \'last_edited\': \'\', \'item_id\': \'211820\', \'helpful\': \'12 of 16 people (75%) found this review helpful\', \'recommend\': True, \'review\': "It\'s like Terraria, you play for 9 hours straight, get endgame armour then stop playing until the next update."}, {\'funny\': \'2 people found this review funny\', \'posted\': \'Posted March 14, 2014.\', \'last_edited\': \'\', \'item_id\': \'730\', \'helpful\': \'5 of 5 people (100%) found this review helpful\', \'recommend\': True, \'review\': \'Hold shift to win, Hold CTRL to lose.\'}, {\'funny\': \'\', \'posted\': \'Posted July 11, 2013.\', \'last_edited\': \'\', \'item_id\': \'204300\', \'helpful

In [9]:
#eliminamos aquellas filas que son iguales
print("Filas duplicadas: ", dfreview.duplicated().sum())
dfreview.drop_duplicates(inplace=True)

Filas duplicadas:  313


In [12]:
#creamos funcion para desanidar diccionarios
def obtener_valor(diccionario, clave):
    if pd.isnull(diccionario):
        return "None"
    if not isinstance(diccionario, dict):
        try:
            diccionario = ast.literal_eval(diccionario)

        except (AttributeError):
            return None
    if isinstance(diccionario, dict):
        return diccionario.get(clave)
    else:
        return None

In [14]:
#Creamos función para desanidar reviews (lista de diccionarios)
def desanidador_ld(lista, clave):
    
    if np.all(pd.isnull(lista)):
        return []
    if not isinstance(lista, list):
        try:
            #lista = json.loads(lista.replace("'", "\"").replace("None", "null"))
            lista = ast.literal_eval(lista)
        except (ValueError, SyntaxError, AttributeError):
            return []
    if isinstance(lista, list):
        valores = []
        for i in lista:
            valor= obtener_valor(i, clave)
            if valor is not None:
                valores.append(valor)
            
        if len(valores) == 0:  # Verificar si la lista está vacía
            return []
        else:
            return valores
    else:
        return []

In [15]:
dfreview["item_id"]=dfreview["reviews"].apply(lambda x: desanidador_ld(x, "item_id"))
dfreview["item_id"]

0                                    [1250, 22200, 43110]
1                                [251610, 227300, 239030]
2        [248820, 370360, 237930, 263360, 107200, 224500]
3         [250320, 20920, 204100, 224600, 207610, 108710]
4                           [211420, 211820, 730, 204300]
                               ...                       
25794                                            [261030]
25795                                               [730]
25796                                               [570]
25797                           [233270, 130, 70, 362890]
25798                                  [273110, 730, 440]
Name: item_id, Length: 25485, dtype: object

In [16]:
dfreview["review"]=dfreview["reviews"].apply(lambda x: desanidador_ld(x, "review"))
dfreview["review"]

0        [Simple yet with great replayability. In my op...
1        [I know what you think when you see this title...
2        [A suitably punishing roguelike platformer.  W...
3        [This game... is so fun. The fight sequences h...
4        [Git gud, It's like Terraria, you play for 9 h...
                               ...                        
25794    [I cried in the end its so sadding ]'; I wish ...
25795    [Gra naprawdę fajna.Ale jest kilka rzeczy do k...
25796                                          [Well Done]
25797    [this is a very fun and nice 80s themed shoote...
25798    [had so much fun plaing this and collecting re...
Name: review, Length: 25485, dtype: object

<h1>Análisis de sentimiento</h1>

In [17]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [18]:
# Descargar recursos necesarios para NLTK 
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jhose\AppData\Roaming\nltk_data...


True

In [19]:
# Función para calcular el sentimiento de una lista de reseñas
def analyze_sentiment(reviews):
    sentiment_list = []  # Lista para almacenar los análisis de sentimiento
    
    if not reviews:
        return [1]  # Valor 1 para listas de reseñas vacías (neutral)
    
    # Utilizar NLTK para análisis de sentimiento (VADER)
    analyzer = SentimentIntensityAnalyzer()
    
    for review in reviews:
        sentiment_scores = analyzer.polarity_scores(review)
        
        # Calcular un valor compuesto de sentimiento
        compound_score = sentiment_scores['compound']
        
        if compound_score >= 0.05:
            sentiment_list.append(2)  # Valor 2 para reseñas positivas
        elif compound_score <= -0.05:
            sentiment_list.append(0)  # Valor 0 para reseñas negativas
        else:
            sentiment_list.append(1)  # Valor 1 para reseñas neutrales
    
    return sentiment_list

In [20]:
dfreview['sentiment_analysis'] = dfreview['review'].apply(lambda review_list: analyze_sentiment(review_list))


In [21]:
dfreview['sentiment_analysis']

0                 [2, 2, 2]
1                 [2, 2, 2]
2        [2, 2, 2, 1, 2, 2]
3        [2, 2, 2, 2, 0, 2]
4              [1, 2, 2, 2]
                ...        
25794                   [2]
25795                   [1]
25796                   [2]
25797          [2, 2, 2, 2]
25798             [2, 2, 2]
Name: sentiment_analysis, Length: 25485, dtype: object

In [22]:
#Borramos las columnas que ya no serán necesarias
dfreview.drop(["reviews", "review"], axis=1, inplace=True)

In [23]:
dfreview.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25485 entries, 0 to 25798
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             25485 non-null  object
 1   user_url            25485 non-null  object
 2   item_id             25485 non-null  object
 3   sentiment_analysis  25485 non-null  object
dtypes: object(4)
memory usage: 995.5+ KB


In [26]:
#generamos el archivo csv limpio
dfreview.to_csv("G:/Mi unidad/SOYHENRY/CIENCIA DE DATOS/CURSO/PROYECTOS/PI1_ML 2.0/DATA_LIMPIA/reviewclean.csv", index=False)